In [1]:
with open('data/text.txt', mode='r', encoding='utf-8') as f:
    data = f.read()

In [2]:
keywords = ['python', 'искусственный интеллект', 'машинное обучение', 'язык интерпретируемый']

In [3]:
from sentence_splitter import SyntaxAnalyzer

sa = SyntaxAnalyzer()

2023-11-24 22:59:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-11-24 22:59:15 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

2023-11-24 22:59:15 INFO: Using device: cuda
2023-11-24 22:59:15 INFO: Loading: tokenize
2023-11-24 22:59:15 INFO: Loading: pos
2023-11-24 22:59:16 INFO: Loading: lemma
2023-11-24 22:59:16 INFO: Loading: depparse
2023-11-24 22:59:17 INFO: Done loading processors!


In [14]:
import re

summary = dict()
sentences = sa.get_sentences(sa(data))

sentences = [sen.lower() for sen in sentences]
keywords = [key.lower() for key in keywords]

for sen in sentences:
    for key in keywords:
        if 500 > len(re.sub(key, '', sen)) > 40 and re.search(key, sen):
            if key.lower() in summary.keys():
                if len(summary[key]) < 4:
                    summary[key].append(sen)
            else:
                summary.update({key: [sen]})

summary

{'python': ['у языка python , пожалуй , один из самых простых и понятных синтаксисов относительно других языков .',
  'код на python , написанный на одной платформе , например , на mac , спокойно запустится на другой , например , на windows .',
  'если говорить об областях применения , то язык python применяется во многих , например , в областях , например , во-первых , это веб-разработка .',
  'аналитикам данных python помогает проверки гипотез , визуализации и обработки данных .']}

In [15]:
' '.join(summary['python'])

'у языка python , пожалуй , один из самых простых и понятных синтаксисов относительно других языков . код на python , написанный на одной платформе , например , на mac , спокойно запустится на другой , например , на windows . если говорить об областях применения , то язык python применяется во многих , например , в областях , например , во-первых , это веб-разработка . аналитикам данных python помогает проверки гипотез , визуализации и обработки данных .'

In [20]:
from pywiktionary import WiktionaryParserFactory

parser_factory = WiktionaryParserFactory(default_language='en')
parser = parser_factory.get_page('python')

In [21]:
parser.parse()

{'meanings': {'noun': {'name': '',
   'meanings': [{'meaning': 'A type of large constricting snake.',
     'examples': []},
    {'meaning': '(vulgar, slang) A penis.', 'examples': []}]}}}

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2").to('cuda:0')

/home/prof_evaluation/miniconda3/envs/mass/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def generate(
    model, tok, text,
    do_sample=True, max_length=150, repetition_penalty=1.2,
    top_k=50, top_p=0.95, temperature=1.5,
    num_beams=8,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

generate(model, tokenizer, 'Веб-сервис это ')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Веб-сервис это \nместо, где можно найти любую информацию, \nв том числе и ту, которая вам нужна.\n\nhttp://www.free-lance.ru/web-services/index.php?option=com_contentview=articleid=736:2011-02-24-12-28catid=1:2012-01-04-18-32Itemid=5<s>\nИтоги 1-го тура конкурса "Бизнес-идея" Дорогие друзья!\n\nМы благодарим всех, кто принял участие в 1-м туре конкурса бизнес-идей "БИЗНЕС-ИДЕЯ".\n']

In [12]:
import wikipedia

wikipedia.summary("Веб-программирование")

"Kateryna Lohvynivna Yushchenko (Ukrainian: Катерина Логвинівна Ющенко, Russian: Екатерина Логвиновна Ющенко, 8 December 1919, Chyhyryn; died 15 August 2001) was a Soviet Ukrainian computer and information research scientist, corresponding member of USSR Academy of Sciences (1976), and member of The International Academy of Computer Science. She developed one of the world's first high-level languages with indirect address in programming (Pointers are analogous to this addressing), called the Address programming language. Over the period of her academic career, Yushchenko supervised 45 Ph.D. students. Further professional achievements include Yushchenko being awarded two USSR State Prizes, The USSR Council of Ministers Prize, The Academician Glushkov Prize, and The Order of Princess Olga. Yushchenko was the first woman in the USSR to become a Doctor of Physical and Mathematical Sciences in programming."

In [ ]:
import requests
import pandas as pd

payload = {
    'source': 'yandex_search',
    'domain': 'com',
    'query': 'что такое python',
    'start_page': 1,
    'pages': 5
}

In [6]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

def generate_answer(model, tokenizer, request: str, system_prompt: str):
    s = f"system\n{system_prompt}</s>\n" + \
        f"<s>user\n{request}</s>\n" + \
        f"<s>bot\n"
    request_tokens = tokenizer(s, return_tensors="pt")    
    # del request_tokens['token_type_ids']
    del request_tokens['attention_mask']
    request_tokens = request_tokens.to(model.device)
    answer_tokens = model.generate(**request_tokens,
                    num_beams=1,
                    top_k=20,
                    temperature=0.6,
                    repetition_penalty=1.2,
                    no_repeat_ngram_size=4,
                    max_new_tokens=96,
                    pad_token_id=tokenizer.eos_token_id)[0]
    answer_tokens = answer_tokens[len(request_tokens[0]):-1]
    answer = tokenizer.decode(answer_tokens).strip()
    return answer

In [7]:
model_name = "gurgutan/saiga2-13b-4bit"
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(model_name, device="cpu")
model.eval()

user_text = "Дай определение термину 'Python'"
answer_text = generate_answer(model, tokenizer, user_text, system_prompt)  
print(answer_text)

Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
The safetensors archive passed at /home/prof_evaluation/.cache/huggingface/hub/models--gurgutan--saiga2-13b-4bit/snapshots/d837ee82a7a385639af4af066b1bfec38bb1d2ac/gptq_model-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not suppo

RuntimeError: Unexpected floating ScalarType in at::autocast::prioritize